In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

PREFIX = '../../input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [2]:
coco = COCO('/opt/ml/input/data/train_all.json')

loading annotations into memory...
Done (t=5.88s)
creating index...
index created!


In [3]:
file_names = []
imag_ids = coco.getImgIds()
len(imag_ids)

3272

In [4]:
categories = coco.loadCats(coco.getCatIds())

In [5]:
 image_ids = coco.getImgIds()
 #coco_annotations = coco.loadAnns(annotations_ids)

In [6]:
len(image_ids)

3272

In [7]:
annotations_ids = coco.getAnnIds(imgIds=image_ids[0], iscrowd=False)

coco_annotations = coco.loadAnns(annotations_ids)

In [8]:
coco_annotations[0]['category_id'], coco_annotations[0]['bbox'] # w, h

(8, [109.0, 150.7, 161.9, 159.8])

In [73]:
def sizetocate(set):
    for i in range(30):
        if set < i  * 500:
            return i
    return 30 

In [78]:
sizetocate(14400)

29

In [102]:
avg = 14579
mid_down = 32* 32
mid_up = 96* 96
#avg = 5000

In [84]:
classes = ["UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [87]:
import numpy as np

In [90]:
arr = np.zeros((12,3))

In [91]:
arr

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
rate = 0
cou = 0
arr = np.zeros(8)
#class_cou =  [0 for i in range(len(classes))]
for i in range(len(image_ids)):
    annotations_ids = coco.getAnnIds(imgIds=image_ids[i], iscrowd=False)

    coco_annotations = coco.loadAnns(annotations_ids)
    for j in range(len(coco_annotations)):
        cou += 1
        rate_tem = coco_annotations[j]['bbox'][2] / coco_annotations[j]['bbox'][3] # w, h
        if rate_tem<=(0.25 + 0.33)/2:
            arr[0] += 1
        elif rate_tem<=(0.33 + 0.5  )/2:
            arr[1] += 1
        elif rate_tem<=(0.50 + 1.0)/2:
            arr[2] += 1
        elif rate_tem<=(1.0 + 2.0)/2:
            arr[3] += 1
        elif rate_tem<=(2.0 + 3.0)/2:
            arr[4] += 1
        elif rate_tem<=(3.0 + 4.0)/2:
            arr[5] += 1
        elif rate_tem<=(4.0 + 5.0)/2:
            arr[6] += 1
        else:
            arr[7] += 1
        #sizetocate(tem_size)
        # if tem_size <= mid_down:
        #     a  += 1
        # elif tem_size <= mid_up:
        #     b += 1
        # else:
        #     c += 1

In [20]:
arr/cou * 100

array([ 3.14393939,  4.88636364, 26.06818182, 42.57575758, 16.21212121,
        4.04166667,  1.42045455,  1.65151515])

In [13]:
rate/cou

1.243741136425897

In [113]:
tem_s = a+b+c
print(a, b,c )
print(f'{a/tem_s:.2f}, {b/tem_s:.2f},{c/tem_s:.2f} ')


4458 10308 11634
0.17, 0.39,0.44 


In [114]:
l = 0
r = 0
#class_cou =  [0 for i in range(len(classes))]
for i in range(len(image_ids)):
    annotations_ids = coco.getAnnIds(imgIds=image_ids[i], iscrowd=False)

    coco_annotations = coco.loadAnns(annotations_ids)
    for j in range(len(coco_annotations)):
        tem_size = coco_annotations[j]['bbox'][2] * coco_annotations[j]['bbox'][3] # w, h
        #sizetocate(tem_size)
        if tem_size <= mid_down:
            arr[coco_annotations[j]['category_id']][0] += 1
        elif tem_size <= mid_up:
            arr[coco_annotations[j]['category_id']][1] += 1
        else:
            arr[coco_annotations[j]['category_id']][2] += 1

In [116]:
brr = np.zeros(12)

In [118]:
l = 0
r = 0
#class_cou =  [0 for i in range(len(classes))]
for i in range(len(image_ids)):
    annotations_ids = coco.getAnnIds(imgIds=image_ids[i], iscrowd=False)

    coco_annotations = coco.loadAnns(annotations_ids)
    for j in range(len(coco_annotations)):
        tem_size = coco_annotations[j]['bbox'][2] * coco_annotations[j]['bbox'][3] # w, h
        #sizetocate(tem_size)
        if tem_size <= mid_down:
            brr[coco_annotations[j]['category_id']] += 1
        elif tem_size <= mid_up:
            brr[coco_annotations[j]['category_id']] += 1
        else:
            brr[coco_annotations[j]['category_id']] += 1

In [130]:
for i in range(11):
    print(f' {classes[i]}  :\t{brr[i]} \t{brr[i]/26400 : .2f}')
 

 UNKNOWN  :	160.0 	 0.01
 General trash  :	2782.0 	 0.11
 Paper  :	9311.0 	 0.35
 Paper pack  :	659.0 	 0.02
 Metal  :	562.0 	 0.02
 Glass  :	610.0 	 0.02
 Plastic  :	3090.0 	 0.12
 Styrofoam  :	1343.0 	 0.05
 Plastic bag  :	7643.0 	 0.29
 Battery  :	63.0 	 0.00
 Clothing  :	177.0 	 0.01


In [107]:
pip install seaborn

     |████████████████████████████████| 285 kB 2.0 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [106]:
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [86]:
s = l + r
print(l, r)
print(l /s , r / s)

14766 11634
0.5593181818181818 0.4406818181818182


In [ ]:

        for idx, a in enumerate(coco_annotations):
            # some annotations have basically no width / height, skip them
            if a['bbox'][2] < 1 or a['bbox'][3] < 1:
                continue
            annotation = np.zeros((1, 5))
            annotation[0, :4] = a['bbox']
            annotation[0, 4] = self.coco_label_to_label(a['category_id'])
            annotations = np.append(annotations, annotation, axis=0)
            
        # transform from [x, y, w, h] to [x1, y1, x2, y2]
        annotations[:, 2] = annotations[:, 0] + annotations[:, 2]
        annotations[:, 3] = annotations[:, 1] + annotations[:, 3]
        
        return annotations
